# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.
%load_ext dotenv
%dotenv
# %reload_ext dotenv

In [2]:
import os
import sys
sys.path.append("../../05_src")

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet/*"))

In [4]:
import dask.dataframe as dd

dd_pr = dd.read_parquet(parquet_files)

In [5]:
dd_pr.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03 00:00:00+00:00,43.463028,51.502148,56.464592,48.193848,56.330471,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04 00:00:00+00:00,40.142929,47.567955,49.266811,46.316166,48.730328,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05 00:00:00+00:00,37.652866,44.617310,47.567955,43.141991,47.389126,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06 00:00:00+00:00,36.219193,42.918453,44.349072,41.577251,44.080830,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07 00:00:00+00:00,39.237453,46.494991,47.165592,42.203148,42.247852,2819626,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# Write your code below.
import numpy as np
dd_feat = (dd_pr.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1), 
        Adj_close_lag_1 = x['Adj Close'].shift(1)
        )
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_close_lag_1'] - 1,
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))

/var/folders/bd/rzstncvd2bqd9ylpjx1pqglc0000gn/T/ipykernel_33462/3976483444.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_pr.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [7]:
dd_feat.head()

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag_1,Adj_close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
HUM,2016-01-04 00:00:00+00:00,168.759323,179.750000,179.880005,176.440002,177.669998,969600,Health Care,Managed Health Care,2016,NaN,NaN,NaN,3.440002
HUM,2016-01-05 00:00:00+00:00,168.646622,179.630005,181.460007,178.500000,180.220001,977000,Health Care,Managed Health Care,2016,179.750000,168.759323,-0.000668,2.960007
HUM,2016-01-06 00:00:00+00:00,165.811295,176.610001,179.240005,175.050003,177.800003,1449800,Health Care,Managed Health Care,2016,179.630005,168.646622,-0.016812,4.190002
HUM,2016-01-07 00:00:00+00:00,164.440536,175.149994,176.660004,173.809998,175.080002,1923100,Health Care,Managed Health Care,2016,176.610001,165.811295,-0.008267,2.850006
HUM,2016-01-08 00:00:00+00:00,162.337570,172.910004,176.330002,172.440002,174.559998,738100,Health Care,Managed Health Care,2016,175.149994,164.440536,-0.012789,3.889999


In [8]:
# Write your code below.
pd_feat = dd_feat.compute()

In [10]:
pd_feat['avg_returns_10d'] = pd_feat['returns'].rolling(10).mean()

In [12]:
pd_feat.head(12)

Price,Date,Adj Close,Close,High,Low,Open,Volume,sector,subsector,year,Close_lag_1,Adj_close_lag_1,returns,hi_lo_range,avg_returns_10d
ticker,,,,,,,,,,,,,,,
HUM,2005-01-03 00:00:00+00:00,26.036839,29.160000,29.860001,28.969999,29.860001,1029100,Health Care,Managed Health Care,2005,NaN,NaN,NaN,0.890001,NaN
HUM,2005-01-04 00:00:00+00:00,25.983263,29.100000,29.590000,29.030001,29.360001,1659200,Health Care,Managed Health Care,2005,29.160000,26.036839,-0.002058,0.559999,NaN
HUM,2005-01-05 00:00:00+00:00,26.519001,29.700001,29.850000,28.920000,29.150000,1886800,Health Care,Managed Health Care,2005,29.100000,25.983263,0.020619,0.930000,NaN
HUM,2005-01-06 00:00:00+00:00,26.465425,29.639999,30.299999,29.610001,29.750000,1638400,Health Care,Managed Health Care,2005,29.700001,26.519001,-0.002020,0.689999,NaN
HUM,2005-01-07 00:00:00+00:00,26.268988,29.420000,29.559999,29.260000,29.389999,1194400,Health Care,Managed Health Care,2005,29.639999,26.465425,-0.007422,0.299999,NaN
HUM,2005-01-10 00:00:00+00:00,27.367245,30.650000,30.660000,29.410000,29.580000,2274000,Health Care,Managed Health Care,2005,29.420000,26.268988,0.041808,1.250000,NaN
HUM,2005-01-11 00:00:00+00:00,27.840481,31.180000,31.340000,30.230000,30.570000,2466100,Health Care,Managed Health Care,2005,30.650000,27.367245,0.017292,1.110001,NaN
HUM,2005-01-12 00:00:00+00:00,28.117281,31.490000,31.549999,31.000000,31.180000,1568800,Health Care,Managed Health Care,2005,31.180000,27.840481,0.009942,0.549999,NaN
HUM,2005-01-13 00:00:00+00:00,28.947676,32.419998,32.680000,31.750000,31.790001,2560700,Health Care,Managed Health Care,2005,31.490000,28.117281,0.029533,0.930000,NaN


In [15]:
# pd_feat.info()

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
    * No, moving average return can also be computed in dask.

+ Would it have been better to do it in Dask? Why?
    * If the dataset could not fit into memory, then we would have to do it in Dask.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [x] Created a branch with the correct naming convention.
- [x] Ensured that the repository is public.
- [x] Reviewed the PR description guidelines and adhered to them.
- [x] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.